In [ ]:
import gpmultipy, scipy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gpmultipy import dataset
from gpmultipy.model import Model
from gpmultipy.prior import Prior
from gpmultipy.kernel import RBF, White
from gpmultipy.freeze import Freezer
from gpmultipy.sampler.slice import Slice
from gpmultipy.kernel import Product, Linear

In [ ]:
%matplotlib inline

In [ ]:
ds = dataset.DataSet('data/normalized/ura3-pq-replicate/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='E')
y = (y-y.mean())/y.std()

dm = np.ones((y.shape[1],1))

In [ ]:
plt.plot(x,y,c='k',alpha=.4); plt.plot(x,y.mean(1),c='r',lw=3);
plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/data-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.8)

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-1.5,1.5,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/residuals-singleReplicate.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,4))

resid = y-y.mean(1)[:,None]
plt.plot(x,abs(resid),c='k',alpha=.4);
plt.plot(x,abs(resid).mean(1),c='r',alpha=1);

A = np.ones((x.shape[0],2))
A[:,1] = x[:,0]
beta,_,_,_ = np.linalg.lstsq(A,abs(resid).mean(1))
plt.plot(x,np.dot(A,beta))

plt.ylim(-.1,1)

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(0,1,.2),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/residual-mean-singleReplicate.pdf",bbox_inches='tight')

# No replicate 

In [ ]:
yKernel = White(1,.1)
k1 = RBF(1,1,1)

# yKernel = White(1,1)
# k1 = RBF(1,1,1)

p = y.shape[1]
dm = np.ones((1,p))

prior = Prior(x,k1,range(dm.shape[0]))

model = Model(x,y,dm)

In [ ]:
prior.kernel.sigma

In [ ]:
k1.K_inv(x)

In [ ]:
mu,cov = prior.functionParameters(model,yKernel)

plt.imshow(cov,interpolation='none')

In [ ]:
plt.plot(scipy.stats.multivariate_normal.rvs(mu,cov))

In [ ]:
cov = k1.K(x,sigma=1e-2) + np.eye(x.shape[0])*1e-4
cov_inv = k1.K_inv(x,sigma=1e-2)
cov_inv2 = np.linalg.inv(cov)

In [ ]:
plt.subplot(121)
plt.imshow(np.dot(cov,cov_inv),interpolation='none')

plt.subplot(122)
plt.imshow(np.dot(cov_inv,cov),interpolation='none')

In [ ]:
plt.subplot(121)
plt.imshow(np.dot(cov,cov_inv2),interpolation='none')

plt.subplot(122)
plt.imshow(np.dot(cov_inv2,cov),interpolation='none')

In [ ]:
cov = k1.K(x,sigma=1e-2) #+ np.eye(x.shape[0])*1e-4
chol = gpmultipy.linalg.jitchol(cov)
cov = np.dot(chol,chol.T)

cov_inv = k1.K_inv(x,sigma=1e-2)
cov_inv2 = np.linalg.inv(cov)

In [ ]:
plt.subplot(121)
plt.imshow(np.dot(cov,cov_inv),interpolation='none')

plt.subplot(122)
plt.imshow(np.dot(cov_inv,cov),interpolation='none')

In [ ]:
plt.subplot(121)
plt.imshow(np.dot(cov,cov_inv2),interpolation='none')

plt.subplot(122)
plt.imshow(np.dot(cov_inv2,cov),interpolation='none')

In [ ]:
y_inv = yKernel.K_inv(x)
f_inv = k1.K_inv(x)

A = dm.shape[1]*y_inv + f_inv

chol_A = gpmultipy.linalg.jitchol(A)
chol_A_inv = np.linalg.inv(chol_A)
A_inv = np.dot(chol_A_inv.T,chol_A_inv)

In [ ]:
plt.subplot(131)
plt.imshow(np.dot(A,A_inv),interpolation='none')

plt.subplot(132)
plt.imshow(np.dot(A_inv,A),interpolation='none')

plt.subplot(133)
plt.imshow(A_inv,interpolation='none')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(A_inv,interpolation='none')

In [ ]:
z = np.power(10,np.linspace(-1,2,50))

# plt.plot(np.log10(z),[kSigmaSlice.loglikelihood(np.log10(a)) for a in z])
# plt.plot(np.log(z),[kSigmaSlice.logdensity_fxn(a) for a in z])
plt.plot(np.log10(z),[prior.loglikelihood(model.beta,sigma=a) for a in z])
plt.plot(np.log10(z),[prior.loglikelihood(model.beta,lengthscale=a) for a in z])
# plt.plot(np.log(z),[kSigmaSlice.prior_logdensity_fxn(a) for a in z])

In [ ]:
ySigmaSlice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,sigma=x),
                    #lambda x: scipy.stats.uniform(1e-6,1e0).logpdf(x),
                    #lambda x: scipy.stats.lognorm(1,loc=1e-2,scale=1e-1).logpdf(x),
                    lambda x: scipy.stats.lognorm(1,scale=1e-0).logpdf(x),
                    #debug_lognorm,
                    .2,5,logspace=True)

kSigmaSlice = Slice('kSigma',
                    lambda x: prior.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    #debug_unif,
                    .2,5,logspace=True)

kLengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

samples = []
freeze = Freezer(yKernel=yKernel,k1=k1,model=model)

In [ ]:
thin = 10
burnin = 0
nsample = 1000

for i in range(nsample):
    prior.sample(model,yKernel)

    k1.sigma = kSigmaSlice.sample(k1.sigma)
    yKernel.sigma = ySigmaSlice.sample(yKernel.sigma)
    k1.lengthscale = kLengthscaleSlice.sample(k1.lengthscale)

    if i % thin == 0 and i > burnin:
        print model.dataLikelihood(yKernel),yKernel.sigma, k1.sigma, k1.lengthscale
        samples.append(freeze.freeze())

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(231)
plt.plot(x,y)

plt.subplot(232)
plt.plot(x,np.array([s['model']['beta'][:,0] for s in samples]).T,c='r',alpha=.5)

plt.subplot(234)
plt.hist(np.log10([s['yKernel']['sigma'] for s in samples]))

plt.subplot(235)
plt.hist(np.log10([s['k1']['sigma'] for s in samples]))

plt.subplot(236)
plt.hist(np.log10([s['k1']['lengthscale'] for s in samples]))

In [ ]:
plt.figure(figsize=(6,4))

# gpfanova.plot.plotFunctionSamples(m_noReplicate.functionSamples(0).values,x=x[:,0])
plt.plot(x[:,0],y,c='k',alpha=.4)

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/model-noReplicate-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))

cov = m_noReplicate.y_k.K(m_noReplicate.x)
ysample = np.dot(m_noReplicate.design_matrix,m_noReplicate.functionSamples(0).values[-1,:][None,:]) +\
            scipy.stats.multivariate_normal.rvs(np.zeros(m_noReplicate.n),cov,size=m_noReplicate.m)

fsample = m_noReplicate.functionMatrix()

plt.plot(x[:,0],ysample.T,c='k',alpha=.4); 
plt.plot(x[:,0],y,c='b',alpha=.4); 
# plt.plot(x[:,0],fsample,c='r',lw=3);

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/model-noReplicate-generative-singleReplicate.pdf",bbox_inches='tight',dpi=300)

# With replicate 

In [ ]:
yKernel = White(1,.1)
k1 = RBF(1,1,1)
k2 = RBF(1,1,1)

p = y.shape[1]
dm = np.zeros((1+p,p))
dm[0,:] = 1
np.fill_diagonal(dm[1:,:],1)

prior = Prior(x,k1,[0])
prior2 = Prior(x,k2,range(1,dm.shape[0]))

model = Model(x,y,dm)

In [ ]:
dm

In [ ]:
ySigmaSlice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,sigma=x),
                    lambda x: scipy.stats.lognorm(1,scale=1e-0).logpdf(x),
                    .2,5,logspace=True)

kSigmaSlice = Slice('kSigma',
                    lambda x: prior.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

kLengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2SigmaSlice = Slice('kSigma',
                    lambda x: prior2.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2LengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior2.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

samples = []
freeze = Freezer(yKernel=yKernel,k1=k1,k2=k2,model=model)

In [ ]:
thin = 10
burnin = 0
nsample = 1000

for i in range(nsample):
    prior.sample(model,yKernel)
    prior2.sample(model,yKernel)
    
    yKernel.sigma = ySigmaSlice.sample(yKernel.sigma)
    k1.sigma = kSigmaSlice.sample(k1.sigma)
    k1.lengthscale = kLengthscaleSlice.sample(k1.lengthscale)
    k2.sigma = k2SigmaSlice.sample(k2.sigma)
    k2.lengthscale = k2LengthscaleSlice.sample(k2.lengthscale)

    if i % thin == 0 and i > burnin:
        print model.dataLikelihood(yKernel),yKernel.sigma, k1.sigma, k1.lengthscale
        samples.append(freeze.freeze())

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(231)
plt.plot(x,y)

plt.subplot(232)
plt.plot(x,np.array([s['model']['beta'][:,0] for s in samples]).T,c='r',alpha=.5)

plt.subplot(233)

cmap = plt.get_cmap("spectral")
colors = [cmap(1.*i/p) for i in range(p)]
for i in range(p):
    plt.plot(x,np.array([s['model']['beta'][:,i+1] for s in samples]).T,c=colors[i],alpha=.5)

plt.subplot(234)
plt.hist(np.log10([s['yKernel']['sigma'] for s in samples]))

plt.subplot(235)
plt.hist(np.log10([s['k1']['sigma'] for s in samples]))

plt.subplot(236)
plt.hist(np.log10([s['k1']['lengthscale'] for s in samples]))

In [ ]:
plt.figure(figsize=(10,6))

ncol = 5

cmap = plt.get_cmap("spectral")
colors = [cmap(1.*i/p) for i in range(p)]

for i in range(p):
    plt.subplot(p/ncol,ncol,i+1)
    
    plt.plot(x,np.array([s['model']['beta'][:,i+1] for s in samples[40:]]).T,c=colors[i],alpha=.5)

In [ ]:
import json

In [ ]:
samplesCopy = [s for s in samples]
for s in samplesCopy:
    s['model']['beta'] = s['model']['beta'].tolist()

In [ ]:
s = json.dumps(samplesCopy)

In [ ]:
ofile = open("technical-replicate-m1.json",'w')
ofile.write(s)
ofile.close()

## Linear Kernel 

In [ ]:
ds = dataset.DataSet('data/normalized/ura3-pq-replicate/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='E')
y = (y-y.mean())/y.std()

x = x + 1e-3

In [ ]:
yKernel = White(1,.1)
k1 = RBF(1,1,1)
k2 = Product(Linear(1,1,1),RBF(1,1,1))

p = y.shape[1]
dm = np.zeros((1+p,p))
dm[0,:] = 1
# dm[1:,1:] = 1
np.fill_diagonal(dm[1:,:],1)

prior = Prior(x,k1,[0])
prior2 = Prior(x,k2,range(1,dm.shape[0]))

In [ ]:
dm

In [ ]:
k2 = Product(Linear(1,1,1),RBF(1,1,1))
cov = k2.K(x)

plt.subplot(121)
plt.imshow(cov,interpolation='none')
plt.colorbar()

plt.subplot(122)
plt.plot(scipy.stats.multivariate_normal.rvs(prior2.mu,cov,size=10).T);

In [ ]:
ds = dataset.DataSet('data/normalized/ura3-pq-replicate/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='E')
y = (y-y.mean())/y.std()

x = x + 1e-3

In [ ]:
yKernel = White(1,.1)
k1 = RBF(1,1,1)
k2 = Product(Linear(1,1,1),RBF(1,1,1))

p = y.shape[1]
dm = np.zeros((1+p,p))
dm[0,:] = 1
# dm[1:,1:] = 1
np.fill_diagonal(dm[1:,:],1)

prior = Prior(x,k1,[0])
prior2 = Prior(x,k2,range(1,dm.shape[0]))

In [ ]:
model = Model(x,y,dm)

In [ ]:
ySigmaSlice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,sigma=x),
                    lambda x: scipy.stats.lognorm(1,scale=1e-0).logpdf(x),
                    .2,5,logspace=True)

kSigmaSlice = Slice('kSigma',
                    lambda x: prior.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

kLengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2SigmaSlice = Slice('kSigma',
                    lambda x: prior2.loglikelihood(model.beta,k1_sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2LengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior2.loglikelihood(model.beta,k1_lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2Sigma2Slice = Slice('kSigma',
                    lambda x: prior2.loglikelihood(model.beta,k2_sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

k2Lengthscale2Slice = Slice('kLengthscale',
                    lambda x: prior2.loglikelihood(model.beta,k2_lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

samples = []
freeze = Freezer(yKernel=yKernel,k1=k1,k2=k2,model=model)

In [ ]:
thin = 10
burnin = 0
nsample = 100

for i in range(nsample):
    prior.sample(model,yKernel)
    prior2.sample(model,yKernel)
    
    yKernel.sigma = ySigmaSlice.sample(yKernel.sigma)
    k1.sigma = kSigmaSlice.sample(k1.sigma)
    k1.lengthscale = kLengthscaleSlice.sample(k1.lengthscale)
    k2.k1.sigma = k2SigmaSlice.sample(k2.k1.sigma)
    k2.k1.lengthscale = k2LengthscaleSlice.sample(k2.k1.lengthscale)
    #k2.k2.sigma = k2Sigma2Slice.sample(k2.k2.sigma)
    k2.k2.lengthscale = k2Lengthscale2Slice.sample(k2.k2.lengthscale)

    if i % thin == 0 and i > burnin:
        print model.dataLikelihood(yKernel),yKernel.sigma, k1.sigma, k1.lengthscale, k2.k1.sigma, k2.k1.lengthscale, k2.k2.lengthscale
        samples.append(freeze.freeze())

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(331)
plt.plot(x,y,c='r')

# plt.subplot(232)
plt.plot(x,np.array([s['model']['beta'][:,0] for s in samples]).T,c='k',alpha=.5)

plt.subplot(332)

cmap = plt.get_cmap("spectral")
colors = [cmap(1.*i/p) for i in range(p)]
for i in range(p):
    plt.plot(x,np.array([s['model']['beta'][:,i+1] for s in samples]).T,c=colors[i],alpha=.5)

plt.subplot(333)
plt.hist(np.log10([s['yKernel']['sigma'] for s in samples]))

plt.subplot(334)
plt.hist(np.log10([s['k1']['sigma'] for s in samples]))

plt.subplot(335)
plt.hist(np.log10([s['k1']['lengthscale'] for s in samples]))

plt.subplot(336)
plt.hist(np.log10([s['k2']['k1']['sigma'] for s in samples]))

plt.subplot(337)
plt.hist(np.log10([s['k2']['k1']['lengthscale'] for s in samples]))

plt.subplot(338)
plt.hist(np.log10([s['k2']['k2']['sigma'] for s in samples]))

plt.subplot(339)
plt.hist(np.log10([s['k2']['k2']['lengthscale'] for s in samples]))

In [ ]:
s['k2']

In [ ]:
plt.figure(figsize=(10,6))

ncol = 5

cmap = plt.get_cmap("spectral")
colors = [cmap(1.*(i+5)/(p+10)) for i in range(p)]

for i in range(p):
    plt.subplot(p/ncol,ncol,i+1)
    
    plt.plot([x.min(),x.max()],[0,0],c='k',lw=3)
    plt.plot(x,np.array([s['model']['beta'][:,i+1] for s in samples[0:]]).T,c=colors[i],alpha=.5)
    
    plt.ylim(-2,2)

In [ ]:
plt.imshow(k2.K(x),interpolation='none')

In [ ]:
cov = k2.K(x)

sample = scipy.stats.multivariate_normal.rvs(prior2.mu,cov,size=10)

plt.plot(x,sample.T);
plt.xlim(x.min(),x.max())

In [ ]:
sample.shape

In [ ]:
model.beta.shape

In [ ]:
plt.plot(x,np.sum(sample.T*sample.T,1));
plt.plot(x,np.sum(model.beta[:,1:]*model.beta[:,1:],1));
plt.xlim(x.min(),x.max())

In [ ]:
k2.freeze()

In [ ]:
cov = k1.K(x)

In [ ]:
sample = scipy.stats.multivariate_normal.rvs(prior2.mu,cov,size=1)

plt.plot(x,sample.T);
plt.xlim(x.min(),x.max())

In [ ]:
samplesCopy = [s for s in samples]
for s in samplesCopy:
    s['model']['beta'] = s['model']['beta'].tolist()

In [ ]:
s = json.dumps(samplesCopy)

In [ ]:
ofile = open("technical-replicate-m2.json",'w')
ofile.write(s)
ofile.close()

# Explicit replicate 

In [ ]:
import patsy

In [ ]:
# dm_effect = np.column_stack((np.ones(y.shape[1]),patsy.contrasts.Helmert().code_without_intercept(range(y.shape[1])).matrix)) 
dm_effect = np.column_stack((np.ones(y.shape[1]),patsy.contrasts.Sum().code_without_intercept(range(y.shape[1])).matrix)) 
dm_effect

In [ ]:
dm_effect.shape

In [ ]:
# m_replicateEffect = gpfanova.base.Base(x,y,designMatrix=dm_effect,priorGroups=[range(dm_effect.shape[1])])
m_replicateEffect = gpfanova.base.Base(x,y,
                                       designMatrix=dm_effect,
                                       priorGroups=[[0]]+[range(1,dm_effect.shape[1])],
                                       bounds = {'y_sigma':(-4,2),
                                                 'prior0_sigma':(-4,2),
                                                 'prior1_sigma':(-4,2),
                                                })

In [ ]:
m_replicateEffect.sample(1000,thin=10)

In [ ]:
plt.figure(figsize=(6,4))

tensor = np.array([m_replicateEffect.functionSamples(i).values.T for i in range(m_replicateEffect.f)])

for r in range(y.shape[1]):
    samp = np.dot(tensor.T,m_replicateEffect.design_matrix[r,:])
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
gpfanova.plot.plotFunctionSamples(m_replicateEffect.functionSamples(0).values,x=x[:,0],c='r')
plt.plot(x[:,0],y,c='k',alpha=.6);

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/model-explicitReplicate-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))

tensor = np.array([m_replicateEffect.functionSamples(i).values.T for i in range(m_replicateEffect.f)])

for r in range(y.shape[1]):
    mu = np.dot(tensor.T,m_replicateEffect.design_matrix[r,:]).mean(0)
    cov = m_replicateEffect.y_k.K(x)
    samp = scipy.stats.multivariate_normal.rvs(mu,cov)
    plt.plot(x[:,0],samp,c='k',alpha=.6)
    #gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
# gpfanova.plot.plotFunctionSamples(m_replicateEffect.functionSamples(0).values,x=x[:,0],c='r')
plt.plot(x[:,0],y,c='b',alpha=.6);

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/model-explicitReplicate-generative-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
mu.shape,cov.shape,x.shape,samp.shape

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(x[:,0],y,c='k',alpha=.6);

tensor = np.array([m_replicateEffect.functionSamples(i).values.T for i in range(m_replicateEffect.f)])

for r in range(y.shape[1]):
    samp = np.dot(tensor.T,m_replicateEffect.design_matrix[r,:])
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])

In [ ]:
plt.figure(figsize=(8,4))

plt.subplot(121)
plt.plot(x,m_replicateEffect.functionMatrix()[:,1:],c='k',alpha=.6)

plt.xlabel("time (normalized)",fontsize=20)
plt.ylabel("OD (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

In [ ]:
params = ['y_sigma','prior0_sigma','prior0_lengthscale','prior1_sigma','prior1_lengthscale']

plt.figure(figsize=(len(params)*4,len(params)*4))
for i,p in enumerate(params):
    plt.subplot(len(params),len(params),i+1+i*len(params))
    plt.title(p)
    plt.plot(m_replicateEffect.parameter_history[p])
    
    for j,p2 in enumerate(params[:i]):
        plt.subplot(len(params),len(params),i+1+i*len(params)-j-1)
        plt.scatter(m_replicateEffect.parameter_history[p2],m_replicateEffect.parameter_history[p])

In [ ]:
m_replicateEffect.kernels

In [ ]:


plt.figure(figsize=(8,4))

plt.subplot(121)
for r in range(y.shape[1]):
    plt.plot(x,np.dot(m_replicateEffect.functionMatrix()[:,1:],m_replicateEffect.design_matrix[r,1:]),
             c='k',alpha=.6)
    
plt.plot(x,y-m_replicateEffect.functionMatrix()[:,0][:,None],
         c='k',alpha=.6)

plt.xlabel("time (normalized)",fontsize=20)
plt.ylabel("OD (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(-1.4,.9)

plt.title("estimated $\\beta_i(\mathbf{x})$",fontsize=20)

cov = m_replicateEffect.kernels[1].K(x)

plt.subplot(122)
for r in range(y.shape[1]*3):
    repSamp = scipy.stats.multivariate_normal.rvs(np.zeros(m_replicateEffect.n),cov)
    repSamp -= m_replicateEffect.functionMatrix()[:,-1]
    plt.plot(x,repSamp,c='k',alpha=.6)
    
plt.ylim(-1.4,.9)
plt.title("simulated $\\beta_i(\mathbf{x})$",fontsize=20)

plt.xlabel("time (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.tight_layout()

# plt.savefig("figures/ura3-pq-replicate/model-randomEffects-replicateSample-singleReplicate.pdf",bbox_inches='tight')

In [ ]:


plt.figure(figsize=(8,4))

plt.subplot(121)
#for r in range(y.shape[1]):
var = np.dot(m_replicateEffect.functionMatrix()[:,1:],m_replicateEffect.design_matrix[:,1:].T).std(1)
plt.plot(x,var,
             c='k',alpha=.6)
    
# plt.plot(x,y-m_replicateEffect.functionMatrix()[:,0][:,None],
#          c='k',alpha=.6)

plt.xlabel("time (normalized)",fontsize=20)
plt.ylabel("OD (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.ylim(-1.4,.9)

plt.title("estimated $\\beta_i(\mathbf{x})$",fontsize=20)

cov = m_replicateEffect.kernels[1].K(x)

plt.subplot(122)
samps = []
for r in range(y.shape[1]*3):
    repSamp = scipy.stats.multivariate_normal.rvs(np.zeros(m_replicateEffect.n),cov)
    repSamp -= m_replicateEffect.functionMatrix()[:,-1]
    samps.append(repSamp)
plt.plot(x,np.array(samps).std(0))

plt.ylim(0,.45)

plt.title("simulated $\\beta_i(\mathbf{x})$",fontsize=20)

plt.xlabel("time (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.tight_layout()

# plt.savefig("figures/ura3-pq-replicate/model-randomEffects-replicateSample-singleReplicate.pdf",bbox_inches='tight')

# Random effect 

In [ ]:
dm_randomeffect = np.column_stack((np.ones(y.shape[1]),np.eye(y.shape[1])))
dm_randomeffect

In [ ]:
m_randomEffect = gpfanova.base.Base(x,y,designMatrix=dm_randomeffect,
                                    priorGroups=[[0],range(1,dm_randomeffect.shape[1])],
                                    bounds={'y_sigma':(-4,0)})

In [ ]:
m_randomEffect.sample(1000,thin=10)

In [ ]:
gpfanova.plot.plotFunctionSamples(m_randomEffect.functionSamples(0).values,x=x[:,0])
plt.plot(x[:,0],y,c='k',alpha=.3);

In [ ]:
gpfanova.plot.plotFunctionSamples(m_randomEffect.functionSamples(0).values,x=x[:,0])
gpfanova.plot.plotFunctionSamples(m_replicateEffect.functionSamples(0).values,x=x[:,0],c='r')
gpfanova.plot.plotFunctionSamples(m_noReplicate.functionSamples(0).values,x=x[:,0],c='g')
plt.plot(x[:,0],y,c='k',alpha=.7);

plt.ylim(-.5,.5)
plt.xlim(.2,.6)

plt.ylim(-2,-1.5)
plt.xlim(0,.2,)

In [ ]:
m_randomEffect.functionSamples(0).values.std(0).mean(),m_noReplicate.functionSamples(0).values.std(0).mean(),m_replicateEffect.functionSamples(0).values.std(0).mean()

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x[:,0],y,c='k',alpha=.6);

for r in range(y.shape[1]):
    gpfanova.plot.plotFunctionSamples(m_randomEffect.functionSamples(0).values+
                                      m_randomEffect.functionSamples(1+r).values,x=x[:,0])
    
gpfanova.plot.plotFunctionSamples(m_randomEffect.functionSamples(0).values,x=x[:,0],c='r')

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

# plt.savefig("figures/ura3-pq-replicate/model-randomEffect-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))
# plt.subplot(121)
# cov = m_noReplicate.y_k.K(m_withReplicate.x)
# ysample = np.dot(m_noReplicate.design_matrix,m_noReplicate.functionSamples(0).values[-1,:][None,:]) +\
#             scipy.stats.multivariate_normal.rvs(np.zeros(m_noReplicate.n),cov,size=m_noReplicate.m)

# fsample = m_noReplicate.functionMatrix()

# plt.plot(ysample.T,c='k',alpha=.4); 
# plt.plot(y,c='b',alpha=.4); 
# plt.plot(fsample,c='r',lw=3);

# plt.subplot(122)

for i in range(y.shape[1]):
    muSamp = m_randomEffect.functionMatrix()[:,0]
    
    cov = m_randomEffect.kernels[1].K(x)
    repSamp = scipy.stats.multivariate_normal.rvs(np.zeros(m_randomEffect.n),cov)
    
    cov = m_randomEffect.y_k.K(x)
    ySamp = muSamp + scipy.stats.multivariate_normal.rvs(repSamp,cov)
    
    plt.plot(x[:,0],ySamp,c='k',alpha=.6)

# cov = m_randomEffect.kernels[1].K(x) + m_randomEffect.y_k.K(x)
# ysample = np.dot(m_randomEffect.design_matrix[:,0][:,None],m_randomEffect.functionSamples(0).values[-1,:][None,:]) +\
#             scipy.stats.multivariate_normal.rvs(np.zeros(m_randomEffect.n),cov,size=m_randomEffect.m)

# fsample = m_noReplicate.functionMatrix()


plt.plot(x[:,0],y,c='b',alpha=.6); 
# plt.plot(ysample.T,c='k',alpha=.7); 
# plt.plot(fsample,c='r',lw=3);

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

# plt.savefig("figures/ura3-pq-replicate/model-randomEffects-generative-singleReplicate.pdf",bbox_inches='tight')

In [ ]:
cov = m_randomEffect.kernels[1].K(x)

plt.figure(figsize=(8,4))

plt.subplot(121)
plt.plot(x,m_randomEffect.functionMatrix()[:,1:],c='k',alpha=.6)

plt.xlabel("time (normalized)",fontsize=20)
plt.ylabel("OD (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(-.9,.6)

plt.title("estimated $\\beta_i(\mathbf{x})$",fontsize=20)

plt.subplot(122)
for r in range(y.shape[1]):
    repSamp = scipy.stats.multivariate_normal.rvs(np.zeros(m_randomEffect.n),cov)
    plt.plot(x,repSamp,c='k',alpha=.6)
    
plt.ylim(-.9,.6)
plt.title("simulated $\\beta_i(\mathbf{x})$",fontsize=20)

plt.xlabel("time (normalized)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.tight_layout()

# plt.savefig("figures/ura3-pq-replicate/model-randomEffects-replicateSample-singleReplicate.pdf",bbox_inches='tight')

In [ ]:
m_randomEffect.paa

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(121)
cov = m_noReplicate.y_k.K(m_withReplicate.x)
ysample = np.dot(m_noReplicate.design_matrix,m_noReplicate.functionSamples(0).values[-1,:][None,:]) +\
            scipy.stats.multivariate_normal.rvs(np.zeros(m_noReplicate.n),cov,size=m_noReplicate.m)

fsample = m_noReplicate.functionMatrix()

plt.plot(ysample.T,c='k',alpha=.4); 
plt.plot(y,c='b',alpha=.4); 
plt.plot(fsample,c='r',lw=3);

plt.subplot(122)
cov = m_randomEffect.kernels[1].K(x) + m_randomEffect.y_k.K(x)
ysample = np.dot(m_randomEffect.design_matrix[:,0][:,None],m_randomEffect.functionSamples(0).values[-1,:][None,:]) +\
            scipy.stats.multivariate_normal.rvs(np.zeros(m_randomEffect.n),cov,size=m_randomEffect.m)

# fsample = m_noReplicate.functionMatrix()


plt.plot(y,c='b',alpha=.4); 
plt.plot(ysample.T,c='k',alpha=.7); 
# plt.plot(fsample,c='r',lw=3);

# plt.savefig("figures/ura3-pq-replicate/model-noReplicate-generative.pdf",bbox_inches='tight')

In [ ]:
params = ['y_sigma','prior0_sigma','prior0_lengthscale','prior1_sigma','prior1_lengthscale']

plt.figure(figsize=(len(params)*4,len(params)*4))
for i,p in enumerate(params):
    plt.subplot(len(params),len(params),i+1+i*len(params))
    plt.title(p)
    plt.plot(m_randomEffect.parameter_history[p])
    
    for j,p2 in enumerate(params[:i]):
        plt.subplot(len(params),len(params),i+1+i*len(params)-j-1)
        plt.scatter(m_randomEffect.parameter_history[p2],m_randomEffect.parameter_history[p])

In [ ]:
ysample, fsample = m_randomEffect.samplePrior()

plt.plot(ysample);
plt.plot(fsample[:,0],c='k',lw=3)

In [ ]:
plt.figure(figsize=(4,4))

fsamp = [m_noReplicate.functionSamples(0).values,
         m_replicateEffect.functionSamples(0).values,
         m_randomEffect.functionSamples(0).values]

ints = [gpfanova.interval.FunctionInterval(fs,.95) for fs in fsamp]
deltas = [(i.ub-i.lb) for i in ints]

plt.boxplot(deltas,labels=['no\neffect','fixed\neffect','random\neffect'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("$\mu(\mathbf{x})$ interval size",fontsize=20,y=1.01)

# plt.savefig("figures/ura3-pq-replicate/model-comparison-mu-singleReplicate.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(4,4))

# fsamp = [m_noReplicate.functionSamples(0).values,
#          m_replicateEffect.functionSamples(0).values,
#          m_randomEffect.functionSamples(0).values]

# ints = [gpfanova.interval.FunctionInterval(fs,.95) for fs in fsamp]
# deltas = [(i.ub-i.lb) for i in ints]

ysigma = [m_noReplicate.parameter_history.y_sigma.values,
         m_replicateEffect.parameter_history.y_sigma.values,
         m_randomEffect.parameter_history.y_sigma.values,]

plt.boxplot(ysigma,labels=['no\neffect','fixed\neffect','random\neffect'])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("$log_{10}(\sigma_y)$",fontsize=20,y=1.01)

plt.ylim(-2.1,-1.2)

# plt.savefig("figures/ura3-pq-replicate/model-comparison-ysigma-singleReplicate.pdf",bbox_inches='tight')

In [ ]:
i = ints[0]

# More data 

In [ ]:
ds = gpfanova.dataset.DataSet('data/ura3-pq-replicate/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Bio','Tech'])

neffect = effect.Bio.unique().shape[0]
for e in range(neffect):
    select = effect.values[:,0] == e
    y[:,select] -= y[0,select].mean()
    
y = (y-y.mean())/y.std()

In [ ]:
select = effect.Tech != 9
y = y[:,np.where(select)[0]]
effect = effect.loc[select,:]

In [ ]:
for b in [6,8,10,12]:
    select = (effect != np.array([[b,0]])).values.any(1)
    
    y = y[:,np.where(select)[0]]
    effect = effect.loc[select,:]

In [ ]:
y.shape

In [ ]:
cmap = plt.get_cmap('spectral')
for i in range(y.shape[1]):
    plt.plot(x,y[:,i],c=cmap(1.*effect.values[i,0]/effect.Bio.unique().shape[0]),alpha=.4);

In [ ]:
plt.figure(figsize=(6,10))

ncol = 3

for i in range(y.shape[1]):
    plt.subplot(neffect/ncol+1,ncol,effect.values[i,0]+1)
    #plt.plot(x,y[:,i],c=cmap(1.*effect.values[i,0]/neffect),alpha=.4);
    plt.plot(x,y[:,i],c=cmap(1.*effect.values[i,1]/10),alpha=.8,label=str(effect.values[i,1]));
    
    plt.ylim(-2,2)
    
for i in range(len(labels[0])):
    plt.subplot(neffect/ncol+1,ncol,i+1)
    #plt.title(labels[0][i])
    
    if i%ncol == 0:
        plt.yticks(np.arange(-1.5,2,1),fontsize=10)
        plt.ylabel("OD\n(normalized)",fontsize=15)
    else:
        plt.yticks(np.arange(-1.5,2,1),fontsize=10)
        
    if i >= 11:
        plt.xticks(np.arange(.2,1,.3),fontsize=10)
        plt.xlabel("time\n(normalized)",fontsize=15)
    else:
        plt.xticks(np.arange(.2,1,.3),fontsize=10)
    
plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/data-multipleReplicates.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(10,6))

ncol = 5

for i in range(y.shape[1]):
    plt.subplot(neffect/ncol+1,ncol,effect.values[i,0]+1)
    #plt.plot(x,y[:,i],c=cmap(1.*effect.values[i,0]/neffect),alpha=.4);
    plt.plot(x,y[:,i],c=cmap(1.*effect.values[i,1]/10),alpha=.8,label=str(effect.values[i,1]));
    
    plt.ylim(-2,2)
    
for i in range(len(labels[0])):
    plt.subplot(neffect/ncol+1,ncol,i+1)
    #plt.title(labels[0][i])
    
    if i%ncol == 0:
        plt.yticks(np.arange(-1.5,2,1),fontsize=10)
        plt.ylabel("OD\n(normalized)",fontsize=15)
    else:
        plt.yticks(np.arange(-1.5,2,1),fontsize=10)
        
    if i >= 9:
        plt.xticks(np.arange(.2,1,.3),fontsize=10)
        plt.xlabel("time\n(normalized)",fontsize=15)
    else:
        plt.xticks(np.arange(.2,1,.3),fontsize=10)
    
plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/data-multipleReplicates-alt.pdf",bbox_inches='tight',dpi=300)

In [ ]:
bioMatrix = np.zeros((y.shape[1],effect.Bio.unique().shape[0]))
for i,e in enumerate(effect.Bio):
#     print i,e
    bioMatrix[i,e] = 1
    
plt.imshow(bioMatrix)

In [ ]:
dm_randomEffect = np.column_stack((np.ones(y.shape[1]),
                                   bioMatrix,
                                   np.eye(y.shape[1])))

In [ ]:
m_randomEffect_bio = gpfanova.base.Base(x,y,designMatrix=dm_randomEffect,
                                        priorGroups=[[0],range(1,bioMatrix.shape[1]+1),range(bioMatrix.shape[1]+1,dm_randomEffect.shape[1])],
                                        bounds={'prior0_lengthscale':(-1,2),
                                                'prior1_lengthscale':(-1,2),
                                                'prior2_lengthscale':(-1,2),
                                                'prior1_sigma':(-4,-1),
                                                'prior2_sigma':(-4,-1),})

m_randomEffect_bio.parameter_cache.prior1_sigma = -2
m_randomEffect_bio.parameter_cache.prior2_sigma = -2

In [ ]:
m_randomEffect_bio.sample(50,thin=10)

In [ ]:
gpfanova.plot.plotFunctionSamples(m_randomEffect_bio.functionSamples(0).values,x=x[:,0])
plt.plot(x[:,0],y,c='k',alpha=.3);

In [ ]:
plt.figure(figsize=(16,16))


tensor = np.array([m_randomEffect_bio.functionSamples(i).values.T for i in range(m_randomEffect_bio.f)])

for r in range(y.shape[1]):
    plt.subplot(neffect/4+1,4,effect.values[r,0]+1)
    samp = np.dot(tensor.T,m_randomEffect_bio.design_matrix[r,:])
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
    plt.plot(x[:,0],y[:,r],c='k',alpha=.6);
    
for e in range(neffect):
    plt.subplot(neffect/4+1,4,e+1)
    samp = m_randomEffect_bio.functionSamples(0).values + m_randomEffect_bio.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c='r')
    
    plt.ylim(-2,2)

In [ ]:
plt.figure(figsize=(16,16))

for e in range(neffect):
    plt.subplot(neffect/4+1,4,e+1)
    samp = m_randomEffect_bio.functionSamples(0).values + m_randomEffect_bio.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c=cmap(1.*e/neffect))
    plt.plot(x[:,0],y[:,effect.values[:,0]==e].mean(1),c=cmap(1.*e/neffect))
    
    gpfanova.plot.plotFunctionSamples(m_randomEffect_bio.functionSamples(0).values,x=x[:,0],c='k')
    
    plt.ylim(-2,2)

In [ ]:
params = ['y_sigma','prior0_sigma','prior0_lengthscale','prior1_sigma','prior1_lengthscale','prior2_sigma','prior2_lengthscale']

plt.figure(figsize=(len(params)*4,len(params)*4))
for i,p in enumerate(params):
    plt.subplot(len(params),len(params),i+1+i*len(params))
    plt.title(p)
    plt.plot(m_randomEffect_bio.parameter_history[p])
    
    for j,p2 in enumerate(params[:i]):
        plt.subplot(len(params),len(params),i+1+i*len(params)-j-1)
        plt.scatter(m_randomEffect_bio.parameter_history[p2],m_randomEffect_bio.parameter_history[p])

In [ ]:
cov = m_randomEffect_bio.kernels[2].K(m_randomEffect.x,prior2_lengthscale=-1.5,prior2_sigma=0)

fsample = scipy.stats.multivariate_normal.rvs(np.zeros(m_randomEffect_bio.x.shape[0]),cov)

plt.plot(x[:,0],fsample)

In [ ]:
m_randomEffect_bio.functionSamples(0).values.std(0).mean(),\
m_randomEffect.functionSamples(0).values.std(0).mean(),\
m_noReplicate.functionSamples(0).values.std(0).mean(),\
m_replicateEffect.functionSamples(0).values.std(0).mean()

## Integrate replicates 

In [ ]:
reload(gpfanova.base)

In [ ]:
bioMatrix = np.zeros((y.shape[1],effect.Bio.unique().shape[0]))
for i,e in enumerate(effect.Bio):
#     print i,e
    bioMatrix[i,e] = 1
    
plt.imshow(bioMatrix)

In [ ]:
dm_randomEffect_intRep = np.column_stack((np.ones(y.shape[1]),
                                   bioMatrix,))

m_randomEffect_bio_intRep = gpfanova.base.Base_withReplicate(x,y,designMatrix=dm_randomEffect_intRep,
                                        priorGroups=[[0],range(1,bioMatrix.shape[1]+1)],
                                        bounds={'y_sigma':(-4,-1),
                                                'prior0_lengthscale':(-1,2),
                                                'prior1_lengthscale':(-1,2),
                                                'prior0_sigma':(-4,-1),
                                                'prior1_sigma':(-4,-1),
                                                },
                                        replicateSigmaBounds=(-4,-1),
                                        replicateLengthscaleBounds=(-1,2))

# m_randomEffect_bio_intRep = gpfanova.base.Base(x,y,designMatrix=dm_randomEffect_intRep,
#                                         priorGroups=[[0],range(1,bioMatrix.shape[1]+1)],
#                                         bounds={'prior0_lengthscale':(-1,2),
#                                                 'prior1_lengthscale':(-1,2),
#                                                 'prior1_sigma':(-4,-1),
#                                                 'y_sigma':(-4,-1)},
#                                         replicateSigmaBounds=(-4,-1),
#                                         replicateLengthscaleBounds=(-1,2))

m_randomEffect_bio_intRep.parameter_cache.prior0_sigma

In [ ]:
m_randomEffect_bio_intRep.sample(500,thin=10)

In [ ]:
gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0])
plt.plot(x[:,0],y,c='k',alpha=.3);

In [ ]:
plt.figure(figsize=(16,16))

tensor = np.array([m_randomEffect_bio_intRep.functionSamples(i).values.T for i in range(m_randomEffect_bio_intRep.f)])

for r in range(y.shape[1]):
    plt.subplot(neffect/4+1,4,effect.values[r,0]+1)
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    
    #gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    samp = samp[-1,:]
    cov = m_randomEffect_bio_intRep.y_k.K(x)

    sample = scipy.stats.multivariate_normal.rvs(samp,cov)

    plt.plot(x[:,0],sample,c='r',alpha=.4)
    
    plt.plot(x[:,0],y[:,r],c='k',alpha=.4);
    
for e in range(neffect):
    plt.subplot(neffect/4+1,4,e+1)
    samp = m_randomEffect_bio_intRep.functionSamples(0).values + m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c='r')
    
    gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0],c='k')
    
    if e >= 10:
        plt.xlabel("time (normalized)",fontsize=20)
        
    if e%4 == 0:
        plt.ylabel("OD\n(normalized)",fontsize=20)
        
    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    
    
    plt.ylim(-2,2)
    
plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/model-multipleReplicates-generative.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(20,12))

tensor = np.array([m_randomEffect_bio_intRep.functionSamples(i).values.T for i in range(m_randomEffect_bio_intRep.f)])

for r in range(y.shape[1]):
    plt.subplot(neffect/3+1,5,effect.values[r,0]+1)
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    
    #gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    samp = samp[-1,:]
    cov = m_randomEffect_bio_intRep.y_k.K(x)

    sample = scipy.stats.multivariate_normal.rvs(samp,cov)

    plt.plot(x[:,0],sample,c='r',alpha=.4)
    
    plt.plot(x[:,0],y[:,r],c='k',alpha=.4);
    
for e in range(neffect):
    plt.subplot(neffect/3+1,5,e+1)
    samp = m_randomEffect_bio_intRep.functionSamples(0).values + m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c='r')
    
    gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0],c='k')
    
    if e >= 9:
        plt.xlabel("time (normalized)",fontsize=20)
        
    if e%5 == 0:
        plt.ylabel("OD\n(normalized)",fontsize=20)
        
    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    
    
    plt.ylim(-2,2)
    
plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/model-multipleReplicates-generative-alt.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(12,4))

tensor = np.array([m_randomEffect_bio_intRep.functionSamples(i).values.T for i in range(m_randomEffect_bio_intRep.f)])

effectSelect = [4,5]

for r in range(y.shape[1]):
    if not effect.values[r,0] in effectSelect:
        continue
    
    plt.subplot(1,2,effectSelect.index(effect.values[r,0])+1)
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    
    #gpfanova.plot.plotFunctionSamples(samp,x=x[:,0])
    
    samp = np.dot(tensor.T,m_randomEffect_bio_intRep.design_matrix[r,:])
    samp = samp[-1,:]
    cov = m_randomEffect_bio_intRep.y_k.K(x)

    sample = scipy.stats.multivariate_normal.rvs(samp,cov)

    plt.plot(x[:,0],sample,c='r',alpha=.4)
    
    plt.plot(x[:,0],y[:,r],c='k',alpha=.4);
    
for e in range(len(effectSelect)):
    plt.subplot(1,2,e+1)
    samp = m_randomEffect_bio_intRep.functionSamples(0).values + m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c='r')
    
    gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0],c='purple')
    
    plt.xlabel("time (normalized)",fontsize=20)
        
    if e%4 == 0:
        plt.ylabel("OD\n(normalized)",fontsize=20)
        
    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    
    
    plt.ylim(-2,2)
    
plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/model-multipleReplicates-generative-select.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(16,16))

for e in range(neffect):
    plt.subplot(neffect/4+1,4,e+1)
    samp = m_randomEffect_bio_intRep.functionSamples(0).values + m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c=cmap(1.*e/neffect))
    plt.plot(x[:,0],y[:,effect.values[:,0]==e].mean(1),c=cmap(1.*e/neffect))
    
    gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0],c='k')
    
    plt.ylim(-2,2)

In [ ]:
plt.figure(figsize=(16,10))

buff = 2
for e in range(neffect):
    samp = m_randomEffect_bio_intRep.functionSamples(0).values + m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c=cmap(1.*(e+buff)/(neffect+2*buff)))
    plt.plot(x[:,0],y[:,effect.values[:,0]==e].mean(1),c=cmap(1.*(e+buff)/(neffect+2*buff)))
    
gpfanova.plot.plotFunctionSamples(m_randomEffect_bio_intRep.functionSamples(0).values,x=x[:,0],c='k')
    
plt.ylim(-2,1.5)

In [ ]:
plt.figure(figsize=(25,10))

for e in range(neffect):
    plt.subplot(neffect/7+1,7,e+1)
    samp = m_randomEffect_bio_intRep.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c=cmap(1.*e/neffect))
    plt.plot(x[:,0],y[:,effect.values[:,0]==e].mean(1)-m_randomEffect_bio_intRep.functionSamples(0).values.mean(0),c=cmap(1.*e/neffect))
    
    if e >= 7:
        plt.xlabel("time (normalized)",fontsize=20)
        
    if e%7 == 0:
        plt.ylabel("OD\n(normalized)",fontsize=20)
        
    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    
    plt.ylim(-1,1)
    
plt.tight_layout()
plt.savefig("figures/ura3-pq-replicate/model-multipleReplicates-bio.pdf",bbox_inches='tight',dpi=300)

In [ ]:
params = ['y_sigma','replicate_lengthscale','replicate_sigma','prior0_sigma','prior0_lengthscale','prior1_sigma','prior1_lengthscale']

plt.figure(figsize=(len(params)*3,len(params)*3))
for i,p in enumerate(params):
    plt.subplot(len(params),len(params),i+1+i*len(params))
    plt.title(p)
    plt.plot(m_randomEffect_bio_intRep.parameter_history[p])
    
    for j,p2 in enumerate(params[:i]):
        plt.subplot(len(params),len(params),i+1+i*len(params)-j-1)
        plt.scatter(m_randomEffect_bio_intRep.parameter_history[p2],m_randomEffect_bio_intRep.parameter_history[p])

In [ ]:
cov = m_randomEffect_bio_intRep.kernels[1].K(m_randomEffect.x,prior1_sigma=-1)

fsample = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]),cov,size=10)

plt.plot(x[:,0],fsample.T);

## Seperate prior for each BIO 

In [ ]:
bounds = {'prior%d_sigma'%i:(-4,-1) for i in range(dm_randomEffect_intRep.shape[1])}
bounds.update({'prior%d_lengthscale'%i:(-1,2) for i in range(dm_randomEffect_intRep.shape[1])})
bounds['y_sigma'] = (-4,-1)

bounds

In [ ]:
# dm_randomEffect_intRep = np.column_stack((np.ones(y.shape[1]),
#                                    bioMatrix,))

m_randomEffect_bio_intRep_seperate = gpfanova.base.Base_withReplicate(x,y,designMatrix=dm_randomEffect_intRep,
                                        priorGroups=[[i] for i in range(dm_randomEffect_intRep.shape[1])],
                                        bounds=bounds,
                                        replicateSigmaBounds=(-4,-1),
                                        replicateLengthscaleBounds=(-1,2))

In [ ]:
m_randomEffect_bio_intRep_seperate.sample(300,thin=10)

In [ ]:
plt.figure(figsize=(16,16))

for e in range(neffect):
    plt.subplot(neffect/4+1,4,e+1)
    samp = m_randomEffect_bio_intRep_seperate.functionSamples(e+1).values
    gpfanova.plot.plotFunctionSamples(samp,x=x[:,0],c=cmap(1.*e/neffect))
    plt.plot(x[:,0],y[:,effect.values[:,0]==e].mean(1)-m_randomEffect_bio_intRep_seperate.functionSamples(0).values.mean(0),c=cmap(1.*e/neffect))
    
    plt.ylim(-1,1)

In [ ]:
params = bounds.keys()
params.sort()

plt.figure(figsize=(len(params)*3,len(params)*3))
for i,p in enumerate(params):
    plt.subplot(len(params),len(params),i+1+i*len(params))
    plt.title(p)
    plt.plot(m_randomEffect_bio_intRep_seperate.parameter_history[p])
    
    for j,p2 in enumerate(params[:i]):
        plt.subplot(len(params),len(params),i+1+i*len(params)-j-1)
        plt.scatter(m_randomEffect_bio_intRep_seperate.parameter_history[p2],m_randomEffect_bio_intRep_seperate.parameter_history[p])

In [ ]:
for i in range(dm_randomEffect_intRep.shape[1]):
    plt.hist(m_randomEffect_bio_intRep_seperate.parameter_history['prior%d_sigma'%i],alpha=.6)

In [ ]:
plt.figure(figsize=(4,4))

k = m_randomEffect.kernels[1].K(x)
plt.plot(x-x[x.shape[0]/2,0],k[x.shape[0]/2,:]/k.max())

plt.ylabel("covariance",fontsize=20)
plt.xlabel("$x-x'$",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.ylim(-.01,1.05)
plt.xlim(-.5,.5)

plt.savefig("figures/ura3-pq-replicate/example-kernel.png",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))

k = m_randomEffect.kernels[1].K(x)
plt.imshow(k/k.max(),cmap='magma',interpolation='none',aspect='auto')

# plt.ylabel("covariance",fontsize=20)
plt.xlabel("$x$",fontsize=20)
plt.ylabel("$x'$",fontsize=20)
plt.xticks(np.arange(x.shape[0]/5,x.shape[0],x.shape[0]/5),[z.round(1)[0] for z in x[x.shape[0]/5::x.shape[0]/5]],fontsize=15)
plt.yticks(np.arange(x.shape[0]/5,x.shape[0],x.shape[0]/5),[z.round(1)[0] for z in x[x.shape[0]/5::x.shape[0]/5]],fontsize=15)
# plt.yticks([])

# plt.ylim(-.01,1.05)
# plt.xlim(-.5,.5)

plt.colorbar()

plt.savefig("figures/ura3-pq-replicate/example-kernel-full.png",bbox_inches='tight',dpi=300)

In [ ]:
fsample = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]),k)
ysample = scipy.stats.multivariate_normal.rvs(fsample,m_randomEffect.y_k.K(x)*1.3,size=3)

plt.figure(figsize=(10,4))

plt.plot(x[:,0],ysample[0,:].T,'r',alpha=.6,label='data')
plt.plot(x[:,0],ysample.T,'r',alpha=.6)
plt.plot(x[:,0],fsample,'k',lw=3,label='function')

plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)

plt.legend(loc='best',fontsize=20)

plt.savefig("figures/ura3-pq-replicate/example-data.png",bbox_inches='tight',dpi=300)

In [ ]:
k = m_randomEffect.kernels[1].K(x,prior1_lengthscale=-.5)

fsample = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]),k,size=3)

dm = np.zeros((6,3))
dm[:,0] = 1
dm[:3,1] = 1
dm[3:,2] = 1

mu = np.dot(dm,fsample)
ysample = np.array([scipy.stats.multivariate_normal.rvs(mu[i,:],m_randomEffect.y_k.K(x)*1.3) for i in range(mu.shape[0])]).T

plt.figure(figsize=(10,4))

# plt.plot(x[:,0],fsample.T,'k',lw=3,label='function')

plt.subplot(132)
plt.plot(x[:,0],fsample[0,:],'k',lw=3,label='function')

plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(ysample.min(),ysample.max())

plt.subplot(131)
plt.plot(x[:,0],fsample[1,:],'b',lw=3,label='function')

plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(ysample.min(),ysample.max())

plt.subplot(133)
plt.plot(x[:,0],fsample[2,:],'g',lw=3,label='function')

plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(ysample.min(),ysample.max())

plt.tight_layout()

plt.savefig("figures/ura3-pq-replicate/example-multiple-functions.png",bbox_inches='tight',dpi=300)

plt.figure(figsize=(10,4))

plt.subplot(121)

plt.plot(x[:,0],ysample[:,:3],'k',alpha=.3)
plt.plot(x[:,0],fsample[:2,:].sum(0),'purple',alpha=1,lw=3)
plt.plot(x[:,0],fsample[0,:],'k',alpha=1,lw=3)
plt.plot(x[:,0],fsample[1,:],'b',alpha=1,lw=3)
plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(ysample.min(),ysample.max())

plt.subplot(122)

plt.plot(x[:,0],ysample[:,3:],'k',alpha=.3)
plt.plot(x[:,0],fsample[[0,2],:].sum(0),'purple',alpha=1,lw=3)
plt.plot(x[:,0],fsample[0,:],'k',alpha=1,lw=3)
plt.plot(x[:,0],fsample[2,:],'g',alpha=1,lw=3)

plt.xlabel("$x$",fontsize=20)
plt.xticks(np.arange(.2,1,.2),fontsize=15)
plt.yticks(fontsize=15)

plt.ylim(ysample.min(),ysample.max())


plt.savefig("figures/ura3-pq-replicate/example-multiple-data.png",bbox_inches='tight',dpi=300)

In [ ]:
x.shape